Title: The Logic of NULL Values
Slug: sql/the-logic-of-null-values
Category: SQL
Tags: SELECT, FROM, WHERE, LIMIT, AS, AND, OR, ORDER BY, ASC, TRUE, FALSE, NULL
Date: 2017-08-28
Modified: 2017-08-28

This is a fairly simple overview of three-value logic and null values in SQL. This guide is based on the information in [this article](https://www.red-gate.com/simple-talk/sql/learn-sql-server/sql-and-the-snare-of-three-valued-logic/), which offers much more depth and other examples.

#### Load ipython-sql extension

In [1]:
# The 2 lines below prevent an error message from being displayed when we run %load_ext sql
import warnings
warnings.filterwarnings('ignore')

%load_ext sql
%config SqlMagic.feedback = False

#### Connect to the database

In [2]:
%%sql

postgresql://localhost/dvdrental

'Connected: None@dvdrental'

#### `TRUE` and `FALSE` values
These work as you might expect if you have programming experience or an understanding of Boolean values.

In [12]:
%%sql

SELECT
    TRUE = TRUE AS true_equals_true
    , TRUE = FALSE AS true_equals_false
    , FALSE = FALSE AS false_equals_false
    , TRUE AND FALSE AS true_and_false
    , TRUE OR FALSE AS true_or_false

true_equals_true,true_equals_false,false_equals_false,true_and_false,true_or_false
True,False,True,False,True


#### `NULL`, the third logical value
In SQL, `TRUE`, `FALSE` and `NULL` are fully separate and distinct. This means that `NULL` equals neither `TRUE` nor `FALSE`. In fact, even `NULL` doesn't equal `NULL`.

In [14]:
%%sql

SELECT
    TRUE = NULL AS true_equals_null
    , FALSE = NULL AS false_equals_null
    , NULL = NULL AS null_equals_null

true_equals_null,false_equals_null,null_equals_null
None,None,None


#### Testing for `NULL` values
If even `NULL` doesn't equal `NULL`, how do we test for it? It's simple: we use `IS NULL` or `IS NOT NULL`.

In [18]:
%%sql

SELECT
    TRUE IS NULL AS true_is_null
    , TRUE IS NOT NULL AS true_is_not_null
    , FALSE IS NULL AS false_is_null
    , FALSE IS NOT NULL AS false_is_not_null
    , NULL IS NULL AS null_is_null
    , NULL IS NOT NULL AS null_is_not_null

true_is_null,true_is_not_null,false_is_null,false_is_not_null,null_is_null,null_is_not_null
False,True,False,True,True,False


#### `NULL` values in practice
We can use our video rental database to see the effects of this. The last day on which the store had videos returned was 2nd September 2005.

In [61]:
%%sql

SELECT
    r.rental_id
    , r.return_date
FROM
    rental r
WHERE
    r.return_date > '2005-09-02'
ORDER BY
    r.return_date ASC
LIMIT
    5

rental_id,return_date
15971,2005-09-02 01:28:33
16040,2005-09-02 02:19:33
16005,2005-09-02 02:35:22


But we know that some videos haven't been returned, so these videos have the value `NULL` for `r.return_date`.

In [60]:
%%sql

SELECT
    count(r.rental_id) AS outstanding_loans
FROM
    rental r
WHERE
    r.return_date IS NULL

outstanding_loans
183


To include oustanding loans in the results, we add one line to our query. This works provided we know that `NULL` is only used for unreturned rentals.

In [62]:
%%sql

SELECT
    r.rental_id
    , r.return_date
FROM
    rental r
WHERE
    r.return_date > '2005-09-02'
    OR r.return_date IS NULL
ORDER BY
    r.return_date ASC
LIMIT
    5

rental_id,return_date
15971,2005-09-02 01:28:33
16040,2005-09-02 02:19:33
16005,2005-09-02 02:35:22
12101,None
11563,None
